In [167]:
import pandas as pd
import numpy as np

In [168]:
df = pd.read_csv("carData.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51783 entries, 0 to 51782
Data columns (total 13 columns):
price             50987 non-null object
miles             42301 non-null object
fuel_type         51655 non-null object
exterior_color    50885 non-null object
interior_color    45102 non-null object
drivetrain        48653 non-null object
transmission      49856 non-null object
engine            49340 non-null object
VIN               51526 non-null object
name              51657 non-null object
sellerAddress     51517 non-null object
id                51783 non-null int64
modelName         51783 non-null object
dtypes: int64(1), object(12)
memory usage: 5.1+ MB


In [169]:
# Replace empty cells with NAN to prepare for removing rows
df['price'].replace('', np.nan, inplace=True)
df['miles'].replace('', np.nan, inplace=True)
df['sellerAddress'].replace('', np.nan, inplace=True)
df['drivetrain'].replace('', np.nan, inplace=True)
df['transmission'].replace('', np.nan, inplace=True)

# Remove rows with empty cells
df.dropna(subset=['price'], inplace=True)
df.dropna(subset=['miles'], inplace=True)
df.dropna(subset=['sellerAddress'], inplace=True)
df.dropna(subset=['drivetrain'], inplace=True)
df.dropna(subset=['transmission'], inplace=True)

df.reset_index(drop=True, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39694 entries, 0 to 39693
Data columns (total 13 columns):
price             39694 non-null object
miles             39694 non-null object
fuel_type         39694 non-null object
exterior_color    39339 non-null object
interior_color    35241 non-null object
drivetrain        39694 non-null object
transmission      39694 non-null object
engine            39626 non-null object
VIN               39692 non-null object
name              39694 non-null object
sellerAddress     39694 non-null object
id                39694 non-null int64
modelName         39694 non-null object
dtypes: int64(1), object(12)
memory usage: 3.9+ MB


In [170]:
# Remove unit from "miles" column and convert column to number
df['miles'] = df['miles'].str.replace(" miles", "")
df['miles'] = df['miles'].str.replace(",", "")
df['miles'] = pd.to_numeric(df['miles'])

In [171]:
# Remove unit from "price" column and convert column to number
df['price'] = df['price'].str.replace("$", "")
df['price'] = df['price'].str.replace(",", "")
df['price'] = pd.to_numeric(df['price'])

In [172]:
# Standardize values in the "transmission" column

# Start by looking at CVT many CVT columns contain "Automatic"
df.loc[df['transmission'].str.contains('CVT', case=False), 'transmission'] = 'CVT'
df.loc[df['transmission'].str.contains('Variable', case=False), 'transmission'] = 'CVT'

# Find automatic variations
df.loc[df['transmission'].str.contains('Automatic', case=False), 'transmission']= 'Auto'
df.loc[df['transmission'].str.contains('Auto', case=False), 'transmission']= 'Auto'
df.loc[df['transmission'].str.strip() == "A", 'transmission'] = 'Auto'
df.loc[df['transmission'].str.strip() == "a", 'transmission'] = 'Auto'
df.loc[df['transmission'].str.contains('A/T', case=False), 'transmission'] = 'Auto'
df.loc[df['transmission'].str.contains('AT', case=False), 'transmission'] = 'Auto'

# Find manual variation assuming all "#-speed" that don't specify "automatic" are manual
df.loc[df['transmission'].str.contains('Manual', case=False), 'transmission']= 'Manual'
df.loc[df['transmission'].str.strip() == "m", 'transmission'] = 'Manual'
df.loc[df['transmission'].str.contains('Speed', case=False), 'transmission']= 'Manual'
df.loc[df['transmission'].str.contains('Dual Shift', case=False), 'transmission']= 'Manual'

# Now that we've mapped as far as possible remove all rows we couldn't map
df = df[df['transmission'].isin(['CVT', 'Auto', 'Manual'])]

df.reset_index(drop=True, inplace=True)

In [176]:
# Standardize the "drivetrain column"

# Find variations of front wheel drive
df.loc[df['drivetrain'].str.contains('Front', case=False), 'drivetrain'] = 'FWD'
df.loc[df['drivetrain'].str.contains('FWD', case=False), 'drivetrain'] = 'FWD'
df.loc[df['drivetrain'].str.contains('4x2', case=False), 'drivetrain'] = 'FWD'
df.loc[df['drivetrain'].str.contains('2WD', case=False), 'drivetrain'] = 'FWD'

# Find variations of 4 wheel drive assuming 4x4 and all wheel are identical
df.loc[df['drivetrain'].str.contains('Four', case=False), 'drivetrain'] = '4WD'
df.loc[df['drivetrain'].str.contains('4', case=False), 'drivetrain'] = '4WD'
df.loc[df['drivetrain'].str.contains('All', case=False), 'drivetrain'] = '4WD'

# Find variations of rear wheel drive
df.loc[df['drivetrain'].str.contains('RWD', case=False), 'drivetrain'] = 'RWD'
df.loc[df['drivetrain'].str.contains('Rear', case=False), 'drivetrain'] = 'RWD'


# Now that we've mapped as far as possible remove all rows we couldn't map
df = df[df['drivetrain'].isin(['FWD', '4WD', 'RWD'])]

df.reset_index(drop=True, inplace=True)

In [177]:
df.head()

,price,miles,fuel_type,exterior_color,interior_color,drivetrain,transmission,engine,VIN,name,sellerAddress,id,modelName
0,25290,6,Hybrid,Oxford White,Charcoal Black,FWD,CVT,4 Cylinder,1FADP5AU6JL100444,New 2018 Ford C-Max Hybrid SE,"2406 N Jefferson Way, Indianola, IA 50125",719242528,C-Max Hybrid
1,12988,51713,Hybrid,Gray,Charcoal Black,FWD,CVT,2.0L I4 16V MPFI DOHC Hybrid,1FADP5BUXDL528553,Used 2013 Ford C-Max Hybrid SEL,"2020 W 20th St S, Newton, IA 50208",753737631,C-Max Hybrid
2,25995,10,Hybrid,White,Black,FWD,CVT,2.0L I4 16V MPFI DOHC Hybrid,1FADP5BU7GL115512,New 2016 Ford C-Max Hybrid SEL,"916 W St S, Grinnell, IA 50112",675439230,C-Max Hybrid
3,20225,2022,Hybrid,Magnetic Metallic,Charcoal Black,FWD,CVT,2.0L I4 16V MPFI DOHC Hybrid,1FADP5AUXJL105758,New 2018 Ford C-Max Hybrid SE,"600 Brooks Rd, Iowa Falls, IA 50126",738620869,C-Max Hybrid
4,9413,48164,Hybrid,Oxford White,Medium Light Stone/Medium Dark Stone,FWD,Auto,4 Cylinder Engine 2.0L,1FADP5AU3DL540626,Used 2013 Ford C-Max Hybrid SE,"215 15th St SW, Mason City, IA 50401",754880903,C-Max Hybrid


In [188]:
# Split car name into it's componenets (new/used) yead make model
splitNameDf = df['name'].str.split(' ', expand=True, n=3)
df['used'] = splitNameDf[0] != 'New'
df['year'] = splitNameDf[1]
df['make'] = splitNameDf[2]

In [ ]:
# Some non Ford mixed in while scraping, remove those
df = df[df['make'] == "Ford"]